AMPLICON ANALYSIS ATTEMPT 2

Qiime2 setup accoording to instructions for mac: https://docs.qiime2.org/2022.2/install/native/#install-qiime-2-within-a-conda-environment

Installed jupyter lab within qiime environment. When you want to use, activate qiime environment and run jupyter lab from there.

conda activate qiime2-2022.2

jupyter lab

This analysis requires that sample_metadata.tsv file is created following instructions: https://docs.qiime2.org/2022.2/tutorials/metadata/

Metadata was validated throuugh google sheets with Keemei and then downloaded from google drive as .tsv as described in the link above.

In [1]:
cd /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2 # change directory

Testing qiime is working:

In [2]:
qiime --help

Usage: qiime [OPTIONS] COMMAND [ARGS]...

  QIIME 2 command-line interface (q2cli)
  --------------------------------------

  To get help with QIIME 2, visit https://qiime2.org.

  To enable tab completion in Bash, run the following command or add it to
  your .bashrc/.bash_profile:

      source tab-qiime

  To enable tab completion in ZSH, run the following commands or add them to
  your .zshrc:

      autoload -Uz compinit && compinit
      autoload bashcompinit && bashcompinit
      source tab-qiime

Options:
  --version   Show the version and exit.
  --help      Show this message and exit.

Commands:
  info                Display information about current deployment.
  tools               Tools for working with QIIME 2 files.
  dev                 Utilities for developers and advanced users.
  alignment           Plugin for generating and manipulating alignments.
  composition         Plugin for compositional data analysis.
  cutadapt            Plugin for removing adapter sequen

In [3]:
source tab-qiime # allow qiime tab completion

Run fastqc and multiqc

In [8]:
fastqc -o /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/fastqc --noextract -f fastq --casava --quiet /Volumes/TOSHIBA_MAC/Data/NuOMICS/20220524_CDurant_16S_with_NuOMICS_controls/*.fastq.gz

In [9]:
multiqc fastqc/ # run multiqc on fastqc folder


  /// ]8;id=961897;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.12

|           multiqc | Search path : /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/fastqc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 568/568  c/A3DDW2_S5_L001_R2_fastqc.htmlfastqc.html
|            fastqc | Found 284 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete
|           multiqc | 7 flat-image plots used in the report due to large sample numbers
|           multiqc | To force interactive plots, use the '--interactive' flag. 
See the ]8;id=144254;https://multiqc.info/docs/#flat--interactive-plots\documentation]8;;\.


Having a look at the multiqc report:

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/multiqc_report.html

- Some adaptor sequences present from base 125 onwards (<1% of sequences)
- poor sequence quality for NC21 (0.3/1), and not great (0.5/1) quality for read two of NC12 and NC11.
- sequence counts vary from <20,000 to >100,000 reads
- Generally negative controls very low reads, and sample A6CWF1. 
- NuOMICS postive control ~ 115,000 reads which is around the highest read counts achieved for samples (~120,000)
- Ants have more of the same overrepresented sequences than nests
- Per tile sequence quality not great for R2 generally but okay for R1, except nuomics negative, NC21 and A6CWF1 (bad for R2, not great for R1).

Importing data

from: https://docs.qiime2.org/2022.2/tutorials/importing/#sequence-data-with-sequence-quality-information-i-e-fastq

Casava 1.8 paired-end demultiplexed fastq
Format description
In Casava 1.8 demultiplexed (paired-end) format, there are two fastq.gz files for each sample in the study, each containing the forward or reverse reads for that sample. The file name includes the sample identifier. The forward and reverse read file names for a single sample might look like L2S357_15_L001_R1_001.fastq.gz and L2S357_15_L001_R2_001.fastq.gz, respectively. The underscore-separated fields in this file name are:

the sample identifier,

the barcode sequence or a barcode identifier,

the lane number,

the direction of the read (i.e. R1 or R2), and

the set number.

In [13]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /Volumes/TOSHIBA_MAC/Data/NuOMICS/20220524_CDurant_16S_with_NuOMICS_controls \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Imported /Volumes/TOSHIBA_MAC/Data/NuOMICS/20220524_CDurant_16S_with_NuOMICS_controls as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza


NOTE: if you have an error like "Quality score length doesn't match sequence length" and the number of characters do appear to be the same, check whether the file is missing a whitespace at the end.

In [14]:
qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

Saved Visualization to: demux-paired-end.qzv


In [22]:
qiime tools view demux-paired-end.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Above we look at the sequence quality based on ten-thousand randomly selected reads from the subsampled and filtered data. Then we'll denoise the data. When you view the quality plots, note that in contrast to the corresponding plots in the moving pictures tutorial, there are now two interactive plots to be considered together. The plot on the left presents the quality scores for the forward reads, and the plot on the right presents the quality scores for the reverse reads. We’ll use these plots to determine what trimming parameters we want to use for denoising with DADA2, and then denoise the reads using dada2 denoise-paired.

Are there primers and adaptors in the reads?
From: https://forum.qiime2.org/t/clarification-on-primers-are-a-portion-of-them-left-in-this-run/11684/2 

You know the adapters and primers must be getting amplified to create useful Illumina libraries, but with a little bit of cleverness during sequencing, they might never end up within your 250 bp reads!

The MiSeq uses several different primer sets during its sequencing by synthesis process. After clusters have been amplified on the flow cell using the Illumina™ primer, you can perform the sequencing by synthesis reading process using the exact same primer you used for initial PCR. This will begin ‘reading’ the DNA after the adapter region, leaving you with no primers or adapters to remove.

UPDATE: Emailed NuOMICS about this and they said: 

"The adaptor sequences will be removed from the reads during demiltiplexing but the gene specific primer sequences will still be included as the decision to remove these from the reads is at the discretion of the end user"

"We use the Earth microbiome project primers which are the updated Parada/Apprill version but can also be found at the link below.
 
https://earthmicrobiome.org/protocols-and-standards/16s/
 
Updated sequences: 515F (Parada)–806R (Apprill), forward-barcoded:
FWD:GTGYCAGCMGCCGCGGTAA; REV:GGACTACNVGGGTWTCTAAT

"



#notes: - trunc-len and -p-timr-lef are tuning parameters and the length should be based on visual examination of the q-score distribution over the read length; maxee and other parameters can also be modifed.

You could try uisng the base pair at which the median quality drops below 30.

In attempt 1 I used the following:

--p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 250 \
  --p-trunc-len-r 182 \
  
In this attempt I will be more stringent so will cut the first 15 bases from the front of each read, which is commonly done, and cut at 180 for F reads and 104 for R reads.

This will give an overlap of 76.

NOTE: primers are ~20bp long and should be removed as they are prone to errors. Next time you could remove 20bp instead of 15 from the start?

https://www.nicholas-ollberding.com/post/denoising-amplicon-sequence-variants-using-dada2-deblur-and-unoise3-with-qiime2/

This step takes quite a long time. 

In [17]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trim-left-f 15 \
  --p-trim-left-r 15 \
  --p-trunc-len-f 180 \
  --p-trunc-len-r 104 \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza \
  --o-denoising-stats denoising-stats.qza

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza


Generate summaries of artifacts for visualisation

In [19]:
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample_metadata.tsv

qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv
Saved Visualization to: denoising-stats.qzv


In [2]:
qiime tools view table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [27]:
qiime tools view rep-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [1]:
qiime tools view denoising-stats.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

All samples had >90% of input pass filtering other than negative extraction only controls (NC21, NC22, NC11, NC12) which ranged from 39.94%-83.88% of input passing filtering. 

Create a BIOM table with taxonomy annotations. A FeatureTable[Frequency] artefact will be exported as a BIOM v2.1.0 formatted file.

In [23]:
qiime tools export \
  --input-path table.qza \
  --output-path figures

Exported table.qza as BIOMV210DirFmt to directory figures


Then export BIOM to TSV

In [24]:
biom convert \
-i figures/feature-table.biom \
-o figures/feature-table.tsv \
--to-tsv

Some packages require your data to be in a consistent order, i.e. the order of your ASVs in the taxonomy table rows to be the same order of ASVs in the columns of your ASV table. It's recommended to clean up your taxonomy file. You can have blank spots where the level of classification was not completely resolved.

From this point, analysis of paired-end read data progresses in the same way as analysis of single-end read data. You can therefore continue your analyses of these data following the steps that you ran in the moving pictures tutorial.

Questions to guide data analysis
Use the following questions to guide your further analyses of these data data.

What value would you choose to pass for --p-sampling-depth? How many samples will be excluded from your analysis based on this choice? Approximately how many total sequences will you be analyzing in the core-metrics-phylogenetic command?

What sample metadata or combinations of sample metadata are most strongly associated with the differences in microbial composition of the samples? Are these associations stronger with unweighted UniFrac or with Bray-Curtis? Based on what you know about these metrics, what does that difference suggest? For exploring associations between continuous metadata and sample composition, the commands qiime metadata distance-matrix in combination with qiime diversity mantel and qiime diversity bioenv will be useful. These were not covered in the Moving Pictures tutorial, but you can learn about them by running them with the --help parameter.

What do you conclude about the associations between continuous sample metadata and the richness and evenness of these samples? For exploring associations between continuous metadata and richness or evenness, the command qiime diversity alpha-correlation will be useful. This was not covered in the Moving Pictures tutorial, but you can learn about it by running it with the --help parameter.

Which categorical sample metadata columns are most strongly associated with the differences in microbial community richness or evenness? Are these differences statistically significant?

In taxonomic composition bar plots, sort the samples by their average soil relative humidity, and visualize them at the phylum level. What are the dominant phyla in these samples? Which phyla increase and which decrease with increasing average soil relative humidity?

What phyla differ in abundance across vegetated and unvegetated sites?

How do your conclusions differ if you skip the subsampling step above, if at all?

Moving pictures tutorial: https://docs.qiime2.org/2022.2/tutorials/moving-pictures/
 Starting from:
 
 Generate a tree for phylogenetic diversity analyses¶
 
 
 QIIME supports several phylogenetic diversity metrics, including Faith’s Phylogenetic Diversity and weighted and unweighted UniFrac. In addition to counts of features per sample (i.e., the data in the FeatureTable[Frequency] QIIME 2 artifact), these metrics require a rooted phylogenetic tree relating the features to one another. This information will be stored in a Phylogeny[Rooted] QIIME 2 artifact. To generate a phylogenetic tree we will use align-to-tree-mafft-fasttree pipeline from the q2-phylogeny plugin.

First, the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our FeatureData[Sequence] to create a FeatureData[AlignedSequence] QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [30]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


You can export the tree as described in: https://docs.qiime2.org/2022.2/tutorials/exporting/ 

In [31]:
qiime tools export \
  --input-path unrooted-tree.qza \
  --output-path figures/exported-tree

Exported unrooted-tree.qza as NewickDirectoryFormat to directory figures/exported-tree


Alpha and beta diversity analysis

QIIME 2’s diversity analyses are available through the q2-diversity plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We’ll first apply the core-metrics-phylogenetic method, which rarefies a FeatureTable[Frequency] to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. The metrics computed by default are:

Alpha diversity

Shannon’s diversity index (a quantitative measure of community richness)

Observed Features (a qualitative measure of community richness)

Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)

Evenness (or Pielou’s Evenness; a measure of community evenness)

Beta diversity

Jaccard distance (a qualitative measure of community dissimilarity)

Bray-Curtis distance (a quantitative measure of community dissimilarity)

unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

An important parameter that needs to be provided to this script is --p-sampling-depth, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For example, if you provide --p-sampling-depth 500, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice by reviewing the information presented in the table.qzv file that was created above. Choose a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible.

Question

View the table.qzv QIIME 2 artifact, and in particular the Interactive Sample Detail tab in that visualization. What value would you choose to pass for --p-sampling-depth? How many samples will be excluded from your analysis based on this choice? How many total sequences will you be analyzing in the core-metrics-phylogenetic command?


If you view the table.qzv and go to the 'interactive sample detail' tab, then you can play around with the sampling depth for different metadata categories. 

In the moving pictures tutorial they chose sampling depth --p-sampling-depth 1103 which retained 34,193 (22.23%) features in 31 (91.18%) samples at the specifed sampling depth.

Here we set the --p-sampling-depth parameter to 1103. This value was chosen based on the number of sequences in the L3S313 sample because it’s close to the number of sequences in the next few samples that have higher sequence counts, and because it is considerably higher (relatively) than the number of sequences in the samples that have fewer sequences. This will allow us to retain most of our samples. The three samples that have fewer sequences will be dropped from the core-metrics-phylogenetic analyses and anything that uses these results. It is worth noting that all three of these samples are “right palm” samples. Losing a disproportionate number of samples from one metadata category is not ideal. However, we are dropping a small enough number of samples here that this felt like the best compromise between total sequences analyzed and number of samples retained.

Note

The sampling depth of 1103 was chosen based on the DADA2 feature table summary. If you are using a Deblur feature table rather than a DADA2 feature table, you might want to choose a different even sampling depth. Apply the logic from the previous paragraph to help you choose an even sampling depth.

Note

In many Illumina runs you’ll observe a few samples that have very low sequence counts. You will typically want to exclude those from the analysis by choosing a larger value for the sampling depth at this stage.

To rarefy or not to rarefy?

In the first attempt analysi I chose a sampling depth of 17806 which Retained 2,403,810 (27.08%) features in 135 (96.43%) samples at the specifed sampling depth. 

Removed the following samples with specified number of features in brackets: 
NC23 (16764), NC12 (7845), NC11 (5743), A6CWF1 (1521), NC21 (73)


This time I will choose:

21175, which retains 2,879,800 (27.70%) features in 136 (95.77%) samples at the specifed sampling depth.

Samples removed and feature counts are: 

NC23	(18031), 
NC12	(8633), 
NC11	(6533), 
A6CWF1	(1647), 
NUNEG	(1091), 
NC21	(73)

However there are problems with rarefaction and you could look into alternatives. Cannot do sampling depth of 0.
https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003531 

In [32]:
qiime tools view table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [33]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

In [63]:
qiime tools view core-metrics-results/unweighted_unifrac_emperor.qzv 

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

unweighted UniFrac distance - a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features

In [35]:
qiime tools view core-metrics-results/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

In [37]:
qiime tools view core-metrics-results/jaccard_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Jaccard distance- a qualitative measure of community dissimilarity

In [39]:
qiime tools view core-metrics-results/bray_curtis_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [72]:
cd ..

Repeat core metrics results after filtering samples by ant and nest:

In [73]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='ant'" \
  --o-filtered-table ant-table.qza

Saved FeatureTable[Frequency] to: ant-table.qza


In [75]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table ant-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/ant-samples

Saved FeatureTable[Frequency] to: core-metrics-results/ant-samples/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/ant-samples/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/ant-samples/weighted_unifrac_pcoa_results.qz

Create visualisation files:

In [89]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/faith-pd-group-significance.qzv
Saved Visualization to: core-metrics-results/ant-samples/evenness-group-significance.qzv


In [90]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/shannon_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/shannon-group-significance.qzv


In [ ]:
qiime tools view core-metrics-results/ant-samples/faith-pd-group-significance.qzv

In [1]:
qiime tools view core-metrics-results/ant-samples/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [2]:
qiime tools view core-metrics-results/ant-samples/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Beta diversity metrics:

A note on different visualisations:

The Bray-Curtis dissimilarity is based on occurrence data (abundance), while the Jaccard distance is based on presence/absence data (does not include abundance information). UniFrac distances take into account the occurrence table and the phylogeny diversity (sequence distance)

In [148]:
qiime tools view core-metrics-results/ant-samples/unweighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [149]:
qiime tools view core-metrics-results/ant-samples/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [150]:
qiime tools view core-metrics-results/ant-samples/jaccard_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [ ]:
qiime tools view core-metrics-results/ant-samples/bray_curtis_emperor.qzv

BETA GROUP SIGNIFICANCE:

In [132]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/ant-samples/unweighted-unifrac-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/unweighted-unifrac-tree-type-significance.qzv


In [140]:
qiime tools view core-metrics-results/ant-samples/unweighted-unifrac-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [133]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/unweighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/unweighted-unifrac-location-significance.qzv


In [141]:
qiime tools view core-metrics-results/ant-samples/unweighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [134]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/ant-samples/weighted-unifrac-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/weighted-unifrac-tree-type-significance.qzv


In [174]:
qiime tools view core-metrics-results/ant-samples/weighted-unifrac-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [135]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/weighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/weighted-unifrac-location-significance.qzv


In [143]:
qiime tools view core-metrics-results/ant-samples/weighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [136]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/ant-samples/jaccard_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/ant-samples/jaccard-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/jaccard-tree-type-significance.qzv


In [144]:
qiime tools view core-metrics-results/ant-samples/jaccard-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [137]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/ant-samples/jaccard_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/jaccard-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/jaccard-location-significance.qzv


In [145]:
qiime tools view core-metrics-results/ant-samples/jaccard-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [138]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/ant-samples/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/ant-samples/bray_curtis-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/bray_curtis-tree-type-significance.qzv


In [175]:
qiime tools view core-metrics-results/ant-samples/bray_curtis-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [139]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/ant-samples/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/bray_curtis-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/bray_curtis-location-significance.qzv


In [147]:
qiime tools view core-metrics-results/ant-samples/bray_curtis-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [1]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/ant-samples/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column nest-no \
  --o-visualization core-metrics-results/ant-samples/bray_curtis-nest-no-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/bray_curtis-nest-no-significance.qzv


In [2]:
qiime tools view core-metrics-results/ant-samples/bray_curtis-nest-no-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [4]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='ant' AND [location]='Wyre Forest'" \
  --o-filtered-table wyre-forest-ant-table.qza

Saved FeatureTable[Frequency] to: wyre-forest-ant-table.qza


In [5]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table wyre-forest-ant-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/ant-samples/wyre-forest

Saved FeatureTable[Frequency] to: core-metrics-results/ant-samples/wyre-forest/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/wyre-forest/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/wyre-forest/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/wyre-forest/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/wyre-forest/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/wyre-forest/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/wyre-forest/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/wyre-forest/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/wyre-forest/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/ant-samples/wyre-forest/unw

In [6]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/wyre-forest/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/wyre-forest/faith_pd-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/wyre-forest/faith_pd-group-significance.qzv


In [10]:
qiime tools view core-metrics-results/ant-samples/wyre-forest/faith_pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [7]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/wyre-forest/shannon_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/wyre-forest/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/wyre-forest/shannon-group-significance.qzv


In [11]:
qiime tools view core-metrics-results/ant-samples/wyre-forest/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [8]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/wyre-forest/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/wyre-forest/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/wyre-forest/evenness-group-significance.qzv


In [12]:
qiime tools view core-metrics-results/ant-samples/wyre-forest/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [13]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='ant' AND [location]='Dymock Wood'" \
  --o-filtered-table dymock-wood-ant-table.qza

Saved FeatureTable[Frequency] to: dymock-wood-ant-table.qza


In [14]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table dymock-wood-ant-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/ant-samples/dymock-wood

Saved FeatureTable[Frequency] to: core-metrics-results/ant-samples/dymock-wood/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/dymock-wood/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/dymock-wood/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/dymock-wood/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/dymock-wood/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/dymock-wood/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/dymock-wood/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/dymock-wood/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/dymock-wood/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/ant-samples/dymock-wood/unw

In [15]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/dymock-wood/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/dymock-wood/faith_pd-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/dymock-wood/faith_pd-group-significance.qzv


In [19]:
qiime tools view core-metrics-results/ant-samples/dymock-wood/faith_pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [16]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/dymock-wood/shannon_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/dymock-wood/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/dymock-wood/shannon-group-significance.qzv


In [20]:
qiime tools view core-metrics-results/ant-samples/dymock-wood/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [18]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/dymock-wood/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/dymock-wood/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/dymock-wood/evenness-group-significance.qzv


In [21]:
qiime tools view core-metrics-results/ant-samples/dymock-wood/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [21]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='ant' AND [tree-type]='Coniferous'" \
  --o-filtered-table coniferous-ant-table.qza

Saved FeatureTable[Frequency] to: coniferous-ant-table.qza


In [22]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table coniferous-ant-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/ant-samples/coniferous

Saved FeatureTable[Frequency] to: core-metrics-results/ant-samples/coniferous/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/coniferous/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/coniferous/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/coniferous/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/coniferous/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/coniferous/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/coniferous/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/coniferous/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/coniferous/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/ant-samples/coniferous/unweighted_un

In [87]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/coniferous/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/coniferous/faith_pd-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/coniferous/faith_pd-group-significance.qzv


In [160]:
qiime tools view core-metrics-results/ant-samples/coniferous/faith_pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [89]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/coniferous/shannon_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/coniferous/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/coniferous/shannon-group-significance.qzv


In [157]:
qiime tools view core-metrics-results/ant-samples/coniferous/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [91]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/coniferous/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/coniferous/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/coniferous/evenness-group-significance.qzv


In [156]:
qiime tools view core-metrics-results/ant-samples/coniferous/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [23]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/coniferous/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/coniferous/unweighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/coniferous/unweighted-unifrac-location-significance.qzv


In [38]:
qiime tools view core-metrics-results/ant-samples/coniferous/unweighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [44]:
qiime tools view core-metrics-results/ant-samples/coniferous/unweighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [24]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/coniferous/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/coniferous/weighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/coniferous/weighted-unifrac-location-significance.qzv


In [164]:
qiime tools view core-metrics-results/ant-samples/coniferous/weighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [45]:
qiime tools view core-metrics-results/ant-samples/coniferous/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [28]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/coniferous/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/coniferous/bray_curtis-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/coniferous/bray_curtis-location-significance.qzv


In [163]:
qiime tools view core-metrics-results/ant-samples/coniferous/bray_curtis-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [49]:
qiime tools view core-metrics-results/ant-samples/coniferous/bray_curtis_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [39]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/coniferous/jaccard_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/coniferous/jaccard-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/coniferous/jaccard-location-significance.qzv


In [40]:
qiime tools view core-metrics-results/ant-samples/coniferous/jaccard-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [50]:
qiime tools view core-metrics-results/ant-samples/coniferous/jaccard_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [30]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='ant' AND [tree-type]='Deciduous'" \
  --o-filtered-table deciduous-ant-table.qza

Saved FeatureTable[Frequency] to: deciduous-ant-table.qza


In [31]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table deciduous-ant-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/ant-samples/deciduous

Saved FeatureTable[Frequency] to: core-metrics-results/ant-samples/deciduous/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/deciduous/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/deciduous/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/deciduous/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/ant-samples/deciduous/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/deciduous/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/deciduous/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/deciduous/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/ant-samples/deciduous/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/ant-samples/deciduous/unweighted_unifrac_pcoa

In [93]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/deciduous/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/deciduous/faith-pd-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/deciduous/faith-pd-group-significance.qzv


In [159]:
qiime tools view core-metrics-results/ant-samples/deciduous/faith-pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [95]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/deciduous/shannon_vector.qza\
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/deciduous/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/deciduous/shannon-group-significance.qzv


In [158]:
qiime tools view core-metrics-results/ant-samples/deciduous/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [97]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/ant-samples/deciduous/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/ant-samples/deciduous/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/ant-samples/deciduous/evenness-group-significance.qzv


In [155]:
qiime tools view core-metrics-results/ant-samples/deciduous/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [36]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/deciduous/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/deciduous/unweighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/deciduous/unweighted-unifrac-location-significance.qzv


In [37]:
qiime tools view core-metrics-results/ant-samples/deciduous/unweighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [43]:
qiime tools view core-metrics-results/ant-samples/deciduous/unweighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [32]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/deciduous/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/deciduous/weighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/deciduous/weighted-unifrac-location-significance.qzv


In [161]:
qiime tools view core-metrics-results/ant-samples/deciduous/weighted-unifrac-location-significance.qzv


Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [47]:
qiime tools view core-metrics-results/ant-samples/deciduous/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [34]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/deciduous/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/deciduous/bray_curtis-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/deciduous/bray_curtis-location-significance.qzv


In [162]:
qiime tools view core-metrics-results/ant-samples/deciduous/bray_curtis-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [48]:
qiime tools view core-metrics-results/ant-samples/deciduous/bray_curtis_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [41]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/ant-samples/deciduous/jaccard_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/ant-samples/deciduous/jaccard-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/ant-samples/deciduous/jaccard-location-significance.qzv


In [42]:
qiime tools view core-metrics-results/ant-samples/deciduous/jaccard-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [51]:
qiime tools view core-metrics-results/ant-samples/deciduous/jaccard_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Now for nest samples:

In [76]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='nest'" \
  --o-filtered-table nest-table.qza

Saved FeatureTable[Frequency] to: nest-table.qza


In [77]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table nest-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/nest-samples

Saved FeatureTable[Frequency] to: core-metrics-results/nest-samples/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/nest-samples/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/nest-samples/weighted_unifrac_pcoa

Create the visualisation files:

In [82]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/faith-pd-group-significance.qzv
Saved Visualization to: core-metrics-results/nest-samples/evenness-group-significance.qzv


In [94]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/shannon_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/shannon-group-significance.qzv


In [3]:
qiime tools view core-metrics-results/nest-samples/faith-pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [100]:
qiime tools view core-metrics-results/nest-samples/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [101]:
qiime tools view core-metrics-results/nest-samples/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

After computing diversity metrics, we can begin to explore the microbial composition of the samples in the context of the sample metadata. This information is present in the sample metadata file that was downloaded earlier.

We’ll first test for associations between categorical metadata columns and alpha diversity data. We’ll do that here for the Faith Phylogenetic Diversity (a measure of community richness) and evenness metrics.


In [40]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv
Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


In [131]:
qiime tools view core-metrics-results/faith-pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [4]:
qiime tools view core-metrics-results/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Beta diversity metrics:

In [111]:
qiime tools view core-metrics-results/nest-samples/unweighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [171]:
qiime tools view core-metrics-results/nest-samples/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [109]:
qiime tools view core-metrics-results/nest-samples/jaccard_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [108]:
qiime tools view core-metrics-results/nest-samples/bray_curtis_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Beta group significance:

In [115]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/nest-samples/unweighted-unifrac-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/unweighted-unifrac-tree-type-significance.qzv


In [116]:
qiime tools view core-metrics-results/nest-samples/unweighted-unifrac-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [117]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/unweighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/unweighted-unifrac-location-significance.qzv


In [118]:
qiime tools view  core-metrics-results/nest-samples/unweighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [119]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/nest-samples/weighted-unifrac-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/weighted-unifrac-tree-type-significance.qzv


In [172]:
qiime tools view core-metrics-results/nest-samples/weighted-unifrac-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [121]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/weighted-unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/weighted-unifrac-location-significance.qzv


In [122]:
qiime tools view core-metrics-results/nest-samples/weighted-unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [125]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/jaccard_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/nest-samples/jaccard-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/jaccard-tree-type-significance.qzv


In [126]:
qiime tools view core-metrics-results/nest-samples/jaccard-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [123]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/jaccard_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/jaccard-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/jaccard-location-significance.qzv


In [124]:
qiime tools view core-metrics-results/nest-samples/jaccard-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [129]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/nest-samples/bray-curtis-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/bray-curtis-tree-type-significance.qzv


In [173]:
qiime tools view core-metrics-results/nest-samples/bray-curtis-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [127]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/bray-curtis-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/bray-curtis-location-significance.qzv


In [128]:
qiime tools view core-metrics-results/nest-samples/bray-curtis-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Filter by coniferous nest samples only:

In [6]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='nest' AND [tree-type]='Coniferous'" \
  --o-filtered-table coniferous-nest-table.qza

Saved FeatureTable[Frequency] to: coniferous-nest-table.qza


Run core metrics tests:

In [7]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table coniferous-nest-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/nest-samples/coniferous

Saved FeatureTable[Frequency] to: core-metrics-results/nest-samples/coniferous/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/coniferous/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/coniferous/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/coniferous/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/coniferous/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/coniferous/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/coniferous/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/coniferous/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/coniferous/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/nest-samples/coniferous/unw

In [75]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/coniferous/faith_pd_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/coniferous/faith_pd_group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/coniferous/faith_pd_group-significance.qzv


In [151]:
qiime tools view core-metrics-results/nest-samples/coniferous/faith_pd_group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [77]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/coniferous/shannon_vector.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/coniferous/shannon_group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/coniferous/shannon_group-significance.qzv


In [152]:
qiime tools view core-metrics-results/nest-samples/coniferous/shannon_group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [79]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/coniferous/evenness_vector.qza\
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/coniferous/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/coniferous/evenness-group-significance.qzv


In [153]:
qiime tools view core-metrics-results/nest-samples/coniferous/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [9]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/coniferous/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/coniferous/bray-curtis-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/coniferous/bray-curtis-location-significance.qzv


In [168]:
qiime tools view core-metrics-results/nest-samples/coniferous/bray-curtis-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [17]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/coniferous/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/coniferous/weighted_unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/coniferous/weighted_unifrac-location-significance.qzv


In [167]:
qiime tools view core-metrics-results/nest-samples/coniferous/weighted_unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [11]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='nest' AND [tree-type]='Deciduous'" \
  --o-filtered-table deciduous-nest-table.qza

Saved FeatureTable[Frequency] to: deciduous-nest-table.qza


In [12]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table deciduous-nest-table.qza \
  --p-sampling-depth 21175 \
  --m-metadata-file sample_metadata.tsv \
  --output-dir core-metrics-results/nest-samples/deciduous

Saved FeatureTable[Frequency] to: core-metrics-results/nest-samples/deciduous/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/deciduous/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/deciduous/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/deciduous/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/nest-samples/deciduous/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/deciduous/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/deciduous/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/deciduous/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/nest-samples/deciduous/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/nest-samples/deciduous/unweighted_un

In [81]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/deciduous/faith_pd_vector.qza\
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/deciduous/faith_pd-group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/deciduous/faith_pd-group-significance.qzv


In [148]:
qiime tools view core-metrics-results/nest-samples/deciduous/faith_pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [83]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/nest-samples/deciduous/shannon_vector.qza\
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/deciduous/shannon-group-significance.qzv

Saved Visualization to: core-metrics-results/nest-samples/deciduous/shannon-group-significance.qzv


In [149]:
qiime tools view core-metrics-results/nest-samples/deciduous/shannon-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [85]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity  core-metrics-results/nest-samples/deciduous/evenness_vector.qza\
  --m-metadata-file sample_metadata.tsv \
  --o-visualization core-metrics-results/nest-samples/deciduous/evenness-group-significance.qzv


Saved Visualization to: core-metrics-results/nest-samples/deciduous/evenness-group-significance.qzv


In [150]:
qiime tools view core-metrics-results/nest-samples/deciduous/evenness-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [13]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/deciduous/bray_curtis_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/deciduous/bray-curtis-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/deciduous/bray-curtis-location-significance.qzv


In [169]:
qiime tools view core-metrics-results/nest-samples/deciduous/bray-curtis-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [19]:
qiime diversity beta-group-significance \
  --i-distance-matrix  core-metrics-results/nest-samples/deciduous/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/nest-samples/deciduous/weighted_unifrac-location-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/nest-samples/deciduous/weighted_unifrac-location-significance.qzv


In [170]:
qiime tools view core-metrics-results/nest-samples/deciduous/weighted_unifrac-location-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Question

Which categorical sample metadata columns are most strongly associated with the differences in microbial community richness? Are these differences statistically significant?

From analysis attempt 1:

Sample type: As expected there are significant differences between sample types ant and nest (p = 1.628048e-13)

There are significant differences between negative controls (NA), and ants and nests (p=2.236653e-02 and 1.766337e-03, respectively).

There were no significant differences in coniferous and decidious samples (p = 0.473350) however there were differences between negative controls and coniferous and decidious samples (p= 0.008060 and 0.024044, respectively). 

Note large differences in sample sizes however. 

There were no significant differences between different locations other than Harlestone Firs adn Negative Control kit 1 (p = 0.043308). 

For Bedford Perlieus, there were significant differences between nests 1 and 2 (p= 0.037373), and 1 and 4 (p = 0.037373). (All decidious)

There were no significant differences between different nests in Dymock Wood. 

There were no significant differences between nests in Harlestone Firs. 

There were no significant differences between nests in Wyre Forest. 


Nests 1, 5 and 6 in Dymock Wood was signifcantly different to the negative controls  (p= 0.019016, p= 0.010515, p= 0.033006).

Nests 1, 4, 5, 6 and 7 in Wyre Forest were significantly different to the negative controls (p= 0.019016, p= 0.033006, p=0.019016, p=0.027486, p=0.019016). 

For Harlestone Firs nests 4 and 5 were significantly different to the negative controls, (p=0.019016, p=0.010515).


For Bedford Perlieus, there were significant difference between nests 2, 3 and 4  (p=0.033006, p= 0.010515, p=0.010515)



Question

Which categorical sample metadata columns are most strongly associated with the differences in microbial community evenness? Are these differences statistically significant?

There are statistical differences between all sample types including between ants and nests (p=1.554478e-13), ants and negative controls (p=9.033345e02) and nests and negative controls (1.766337e-03)

There are significant differences between coniferous and decidious trees (p=0.000157) but not beween coniferous or decidious trees and negative controls.

There are no significant differences between locations.

There are no significant differences between different nests in Bedford Perlieus.

There was a significant difference between nests 1 and 5 in Dymock Wood (p=0.037373).

There were significant differences between Harlestone Firs 2 and 3 (p=0.037373), 3 and 4 (p=0.037373) and 3 and 5 (p=0.037373)

There were no significant differences between nests in Wyre Forest.




Next we’ll analyze sample composition in the context of categorical metadata using PERMANOVA (first described in Anderson (2001)) using the beta-group-significance command. The following commands will test whether distances between samples within a group, such as samples from the same body site (e.g., gut), are more similar to each other then they are to samples from the other groups (e.g., tongue, left palm, and right palm). If you call this command with the --p-pairwise parameter, as we’ll do here, it will also perform pairwise tests that will allow you to determine which specific pairs of groups (e.g., tongue and gut) differ from one another, if any. This command can be slow to run, especially when passing --p-pairwise, since it is based on permutation tests. So, unlike the previous commands, we’ll run beta-group-significance on specific columns of metadata that we’re interested in exploring, rather than all metadata columns to which it is applicable. Here we’ll apply this to our unweighted UniFrac distances, using two sample metadata columns, as follows.


In [43]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization core-metrics-results/unweighted-unifrac-tree-type-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-tree-type-significance.qzv


In [114]:
qiime tools view core-metrics-results/unweighted-unifrac-tree-type-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

From analysis attempt 1:

There is a significant difference between coniferous and decidious samples (p=0.002), and both coniferous samples and negative controls (p=0.007) and decdidious samples and negative controls (p=0.011)

In [45]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization core-metrics-results/unweighted-unifrac-location-group-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-location-group-significance.qzv


In [113]:
qiime tools view core-metrics-results/unweighted-unifrac-location-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

From analysis attempt 1:

There were significant differences between all locations other than the two negative control kits, Bedford Perlieus and Negative Control Kit 1.

In [47]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column sample-type \
  --o-visualization core-metrics-results/unweighted-unifrac-sample-type-group-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-sample-type-group-significance.qzv


In [48]:
qiime tools view core-metrics-results/unweighted-unifrac-sample-type-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

From analysis attempt 1:

There were significant differences between all sample types (p=0.001).

In [49]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column nest-no \
  --o-visualization core-metrics-results/unweighted-unifrac-nest-no-group-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-nest-no-group-significance.qzv


In [5]:
qiime tools view core-metrics-results/unweighted-unifrac-nest-no-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Alpha rarefaction plotting

In this section we’ll explore alpha diversity as a function of sampling depth using the qiime diversity alpha-rarefaction visualizer. This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with --p-min-depth) and the value provided as --p-max-depth. At each sampling depth step, 10 rarefied tables will be generated, and the diversity metrics will be computed for all samples in the tables. The number of iterations (rarefied tables computed at each sampling depth) can be controlled with --p-iterations. Average diversity values will be plotted for each sample at each even sampling depth, and samples can be grouped based on metadata in the resulting visualization if sample metadata is provided with the --m-metadata-file parameter.

NOTE: HOW TO CHOOSE MAX-DEPTH?

Note

The value that you provide for --p-max-depth should be determined by reviewing the “Frequency per sample” information presented in the table.qzv file that was created above. In general, choosing a value that is somewhere around the median frequency seems to work well, but you may want to increase that value if the lines in the resulting rarefaction plot don’t appear to be leveling out, or decrease that value if you seem to be losing many of your samples due to low total frequencies closer to the minimum sampling depth than the maximum sampling depth.

The visualization will have two plots. The top plot is an alpha rarefaction plot, and is primarily used to determine if the richness of the samples has been fully observed or sequenced. If the lines in the plot appear to “level out” (i.e., approach a slope of zero) at some sampling depth along the x-axis, that suggests that collecting additional sequences beyond that sampling depth would not be likely to result in the observation of additional features. If the lines in a plot don’t level out, this may be because the richness of the samples hasn’t been fully observed yet (because too few sequences were collected), or it could be an indicator that a lot of sequencing error remains in the data (which is being mistaken for novel diversity).

The bottom plot in this visualization is important when grouping samples by metadata. It illustrates the number of samples that remain in each group when the feature table is rarefied to each sampling depth. If a given sampling depth d is larger than the total frequency of a sample s (i.e., the number of sequences that were obtained for sample s), it is not possible to compute the diversity metric for sample s at sampling depth d. If many of the samples in a group have lower total frequencies than d, the average diversity presented for that group at d in the top plot will be unreliable because it will have been computed on relatively few samples. When grouping samples by metadata, it is therefore essential to look at the bottom plot to ensure that the data presented in the top plot is reliable.

67,184.5 is the median frequency in table.qzv

In second attempt median frequency is 78,678.5

Retained 5,586,138 (53.73%) features in 71 (50.00%) samples at the specifed sampling depth.

NOTE: How is this different from the p-sampling-depth chosen earlier in qiime diversity core-metrics-phylogenetic. Should it be the same?


In [2]:
qiime tools view table.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [52]:
qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 78678 \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

Saved Visualization to: alpha-rarefaction.qzv


In [1]:
qiime tools view /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/alpha-rarefaction.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [6]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --p-where "[sample-type]='ant'" \
  --m-metadata-file sample_metadata.tsv \
  --o-filtered-table ant-table.qza 

Saved FeatureTable[Frequency] to: ant-table.qza


Above - filtering samples to only look at ants
where : Str, optional
SQLite WHERE clause specifying sample metadata criteria that must be met to be included in the filtered feature table. If not provided, all samples in metadata that are also in the feature table will be retained.

In [7]:
qiime diversity alpha-rarefaction \
  --i-table ant-table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 78678 \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization ant-alpha-rarefaction.qzv

Saved Visualization to: ant-alpha-rarefaction.qzv


In [145]:
qiime tools view ant-alpha-rarefaction.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Now filter for nest samples:

In [ ]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --p-where "[sample-type]='nest'" \
  --m-metadata-file sample_metadata.tsv \
  --o-filtered-table nest-table.qza 

Complete the rarefaction:

In [143]:
qiime diversity alpha-rarefaction \
  --i-table nest-table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 78678 \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization nest-alpha-rarefaction.qzv

Saved Visualization to: nest-alpha-rarefaction.qzv


In [144]:
qiime tools view nest-alpha-rarefaction.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Taxonomic analysis

In the next sections we’ll begin to explore the taxonomic composition of the samples, and again relate that to sample metadata. The first step in this process is to assign taxonomy to the sequences in our FeatureData[Sequence] QIIME 2 artifact. We’ll do that using a trained Naive Bayes classifier and the q2-feature-classifier plugin. 

Taxonomic classifiers perform best when they are trained based on your specific sample preparation and sequencing parameters, including the primers that were used for amplification and the length of your sequence reads. Therefore in general you should follow the instructions in Training feature classifiers with q2-feature-classifier to train your own taxonomic classifiers. We provide some common classifiers on our data resources page, including Silva-based 16S classifiers, though in the future we may stop providing these in favor of having users train their own classifiers which will be most relevant to their sequence data.

Training feature classifiers with q2-feature-classifier

https://docs.qiime2.org/2022.2/tutorials/feature-classifier/ 

Taxonomic classifiers perform best when they are trained based on your specific sample preparation and sequencing parameters, including the primers that were used for amplification and the length of your sequence reads. Therefore in general you should follow the instructions in Training feature classifiers with q2-feature-classifier to train your own taxonomic classifiers (for example, from the marker gene reference databases in the link below).

https://docs.qiime2.org/2022.2/data-resources/


I have chosen to use SILVA as it is the most comprehensive database and better for environmental, non human analyses? These are already formatted for qiime 2.

Silva 138 SSURef NR99 515F/806R region sequences:
https://data.qiime2.org/2022.2/common/silva-138-99-seqs-515-806.qza (MD5: a914837bc3f8964b156a9653e2420d22)


Silva 138 SSURef NR99 515F/806R region taxonomy 
https://data.qiime2.org/2022.2/common/silva-138-99-tax-515-806.qza (MD5: e2c40ae4c60cbf75e24312bb24652f2c)

Please cite the following references if you use any of these pre-formatted files:

Michael S Robeson II, Devon R O’Rourke, Benjamin D Kaehler, Michal Ziemski, Matthew R Dillon, Jeffrey T Foster, Nicholas A Bokulich. RESCRIPt: Reproducible sequence taxonomy reference database management for the masses. bioRxiv 2020.10.05.326504; doi: https://doi.org/10.1101/2020.10.05.326504


In [55]:
cd training-feature-classifier

In [56]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/training-feature-classifier


In [58]:
wget \
  -O "silva-138-99-seqs-515-806.qza" \
  "https://data.qiime2.org/2022.2/common/silva-138-99-seqs-515-806.qza"


--2022-07-18 18:56:52--  https://data.qiime2.org/2022.2/common/silva-138-99-seqs-515-806.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://qiime2-data.s3-us-west-2.amazonaws.com/2022.2/common/silva-138-99-seqs-515-806.qza [following]
--2022-07-18 18:56:53--  https://qiime2-data.s3-us-west-2.amazonaws.com/2022.2/common/silva-138-99-seqs-515-806.qza
Resolving qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)... 52.92.147.226
Connecting to qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)|52.92.147.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14620394 (14M) [binary/octet-stream]
Saving to: ‘silva-138-99-seqs-515-806.qza’

silva-138-99-seqs-5 100%[===================>]  13.94M  6.66MB/s    in 2.1s    

2022-07-18 18:56:56 (6.66 MB/s) - ‘silv

In [59]:
wget \
  -O "silva-138-99-tax-515-806.qza" \
  "https://data.qiime2.org/2022.2/common/silva-138-99-tax-515-806.qza"

--2022-07-18 18:57:11--  https://data.qiime2.org/2022.2/common/silva-138-99-tax-515-806.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://qiime2-data.s3-us-west-2.amazonaws.com/2022.2/common/silva-138-99-tax-515-806.qza [following]
--2022-07-18 18:57:12--  https://qiime2-data.s3-us-west-2.amazonaws.com/2022.2/common/silva-138-99-tax-515-806.qza
Resolving qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)... 52.92.211.26
Connecting to qiime2-data.s3-us-west-2.amazonaws.com (qiime2-data.s3-us-west-2.amazonaws.com)|52.92.211.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5580678 (5.3M) [binary/octet-stream]
Saving to: ‘silva-138-99-tax-515-806.qza’

silva-138-99-tax-51 100%[===================>]   5.32M  3.36MB/s    in 1.6s    

2022-07-18 18:57:14 (3.36 MB/s) - ‘silva-138-

Extract reference reads

It has been shown that taxonomic classification accuracy of 16S rRNA gene sequences improves when a Naive Bayes classifier is trained on only the region of the target sequences that was sequenced (Werner et al., 2012). This may not necessarily generalize to other marker genes (see note on fungal ITS classification below). We know from the Moving Pictures tutorial that the sequence reads that we’re trying to classify are 120-base single-end reads that were amplified with the 515F/806R primer pair for 16S rRNA gene sequences. We optimize for that here by extracting reads from the reference database based on matches to this primer pair, and then slicing the result to 120 bases.

These were the sequences used by NuOMICS:
Updated sequences: 515F (Parada)–806R (Apprill), forward-barcoded:
FWD:GTGYCAGCMGCCGCGGTAA; REV:GGACTACNVGGGTWTCTAAT


NOTE - do I need to slice for 120? Mine are 250 bp? See below - no.

Note

The --p-trunc-len parameter should only be used to trim reference sequences if query sequences are trimmed to this same length or shorter. Paired-end sequences that successfully join will typically be variable in length. Single-end reads that are not truncated at a specific length may also be variable in length. For classification of paired-end reads and untrimmed single-end reads, we recommend training a classifier on sequences that have been extracted at the appropriate primer sites, but are not trimmed.

Note

The primer sequences used for extracting reads should be the actual DNA-binding (i.e., biological) sequence contained within a primer construct. It should NOT contain any non-biological, non-binding sequence, e.g., adapter, linker, or barcode sequences. If you are not sure what section of your primer sequences are actual DNA-binding, you should consult whoever constructed your sequencing library, your sequencing center, or the original source literature on these primers. If your primer sequences are > 30 nt long, they most likely contain some non-biological sequence.

Note

The example command uses the min-length and max-length parameters to exclude simulated amplicons that are far outside of the anticipated length distribution using those primers. Such amplicons are likely non-target hits and should be excluded. If you adapt this command for your own use, be sure to select settings that are appropriate for the marker gene, not the settings used here. The min-length parameter is applied _after_ the trim-left and trunc-len parameters, and max-length _before_, so be sure to set appropriate settings to prevent valid sequences from being filtered out.

In [60]:
qiime feature-classifier extract-reads \
  --i-sequences silva-138-99-seqs-515-806.qza \
  --p-f-primer GTGYCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACNVGGGTWTCTAAT \
  --p-min-length 100 \
  --p-max-length 400 \
  --o-reads ref-seqs.qza

Saved FeatureData[Sequence] to: ref-seqs.qza


Train the classifier

We can now train a Naive Bayes classifier as follows, using the reference reads and taxonomy that we just created.

In [61]:
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads ref-seqs.qza \
  --i-reference-taxonomy silva-138-99-tax-515-806.qza \
  --o-classifier classifier.qza

Saved TaxonomicClassifier to: classifier.qza


Run the classifier

Finally, we run the classifier and visualize the resulting taxonomic assignments.

In [62]:
qiime feature-classifier classify-sklearn \
  --i-classifier classifier.qza \
  --i-reads ../rep-seqs.qza \
  --o-classification taxonomy.qza

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved FeatureData[Taxonomy] to: taxonomy.qza
Saved Visualization to: taxonomy.qzv


In [64]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/training-feature-classifier


In [125]:
qiime tools view /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/training-feature-classifier/taxonomy.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Question

Recall that our rep-seqs.qzv visualization allows you to easily BLAST the sequence associated with each feature against the NCBI nt database. Using that visualization and the taxonomy.qzv visualization created here, compare the taxonomic assignments with the taxonomy of the best BLAST hit for a few features. How similar are the assignments? If they’re dissimilar, at what taxonomic level do they begin to differ (e.g., species, genus, family, …)?

In [67]:
cd .. # change directory back to main analysis folder

In [127]:
qiime tools view /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/rep-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.

In [69]:
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

Saved Visualization to: taxa-bar-plots.qzv


In [52]:
qiime tools view /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [54]:
qiime taxa barplot \
  --i-table coniferous-ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization coniferous-ant-taxa-bar-plots.qzv

Saved Visualization to: coniferous-ant-taxa-bar-plots.qzv


In [67]:
qiime tools view coniferous-ant-taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [55]:
qiime taxa barplot \
  --i-table deciduous-ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization deciduous-ant-taxa-bar-plots.qzv

Saved Visualization to: deciduous-ant-taxa-bar-plots.qzv


In [64]:
qiime tools view deciduous-ant-taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [56]:
qiime taxa barplot \
  --i-table coniferous-nest-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization coniferous-nest-taxa-bar-plots.qzv

Saved Visualization to: coniferous-nest-taxa-bar-plots.qzv


In [65]:
qiime tools view coniferous-nest-taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [61]:
qiime taxa barplot \
  --i-table deciduous-nest-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization deciduous-nest-taxa-bar-plots.qzv

Saved Visualization to: deciduous-nest-taxa-bar-plots.qzv


In [66]:
qiime tools view deciduous-nest-taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Barplots for negative controls:

In [68]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --p-where "[sample-type]='NA'" \
  --m-metadata-file sample_metadata.tsv \
  --o-filtered-table control-table.qza 

Saved FeatureTable[Frequency] to: control-table.qza


In [69]:
qiime taxa barplot \
  --i-table control-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization control-taxa-bar-plots.qzv

Saved Visualization to: control-taxa-bar-plots.qzv


In [70]:
qiime tools view control-taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Added after and NOT RUN in the first attempt: (something to consider?)
You may want to filter to remove non-bacteria sequences? 

You would do this after taxonomy assignment.

See: https://www.melbournebioinformatics.org.au/tutorials/tutorials/qiime2/qiime2/

According to QIIME developer Nicholas Bokulich, low abundance filtering (i.e. removing ASVs containing very few sequences) is not necessary under the ASV model.

E.g. you might want to filter out reads classified as mitochondria and chloroplast. Unassigned ASVs are retained. Generate a viewable summary file of the new table to see the effect of filtering.

NOTE: in attempt 2 with the changes made, no sequences have been identified as eukaryotic so not neccessary to do this filtering step.

Note

You might notice that some features do not have taxonomic assignments, which for the database is indicated by a blank string at the level (e.g., "g__"). These indicate that there is not enough information for the database to differentiate members of that clade, either due to ambiguity in the database or because the gene region being sequenced doesn’t provide the resolution to distinguish members of that clade. This is distinct from cases where q2-feature-classifier cannot reliably classify the ASV to a deeper level: in those cases, an incomplete taxonomy string will be provided. Hence, you may see two different types of “underclassification” in your data: e.g., k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Christensenellaceae; g__; s__ (genus and species annotations are missing in Greengenes) as well as k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Christensenellaceae (q2-feature-classifier could not confidently classify that ASV at genus level).


Note

You may also notice that more than one ASV has the same taxonomic assignment. This is normal — unique ASVs do not necessarily map to unique taxonomic groups! We visualize the frequency of each taxonomic group in barplots (as described below) or use the q2-taxa plugin to collapse our feature table based on taxonomic affiliation.

Problem solving unassigned taxonomy: 

https://forum.qiime2.org/t/improving-taxonomic-classification/19820/5

Sometimes sequences wind up in mixed orientations, i.e., a single file can contain reads from both the forward and reverse strands of a DNA sequence. This is most common in query sequences derived from non-directionalized library preparation methods, but is occasionally seen in reference sequence collections that are compiled from different sources and/or have not been aligned. The orient-seqs method can orient these sequences by comparison to a reference sequence set known to be oriented in the correct direction.


Any sequences that match are output in the correct(ed) orientation are saved to oriented-query-sequences.qza. Any sequences that do not match any sequences in the reference (and hence cannot be oriented) are saved to unmatched-sequences.qza for inspection. Note that this method has various parameters to control the alignment similarity thresholds (as orientation is performed by using VSEARCH global alignment under the hood).

If this is the case, I'll probably need to rerun the pipleline:

"Your diversity metrics have been impacted, and you should re-orient, dereplicate, and then re-run. Two examples for you to consider:

observed-features is a count of the number of unique features in your data. If a feature is present in your data in both "forward" and "reversed" orientation, it would show up twice, potentially doubling the alpha diversity on this metric.

Any phylogenetic measure (e.g. Unifrac, Faith's PD) will be based on a Phylogeny, almost certainly constructed from your representative sequences. Phylogeny builders won't know some of those sequences are "mis-oriented", and this could result in the construction of a tree that is not meaningful.

So, after the orient-seqs step and before you re-run, you should also dereplicate your ref-seqs and table. You can do this by clustering your features de novo at 100% identity (cluster-features-de-novo can do this for you). This will ensure that any duplicates in your re-aligned rep-seqs are removed, and their counts are combined in your feature table. Once this is done, you should be able to rebuild a more meaningful Phylogeny, and rerun core-metrics-phylogenetic and any downstream analysis."


Install the plugin following the instructions at:

https://github.com/bokulich-lab/RESCRIPt/

Option 2: Install within QIIME 2 environment
First activate your QIIME 2 environment (ver 2022.2 or later) and install relevant dependencies:

conda activate qiime2-2022.2
conda install -c conda-forge -c bioconda -c qiime2 -c defaults xmltodict
Install source:

pip install git+https://github.com/bokulich-lab/RESCRIPt.git


https://forum.qiime2.org/t/improving-taxonomic-classification/19820/5 

In [176]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2


In [1]:
qiime rescript orient-seqs \
	--i-sequences rep-seqs.qza \
	--i-reference-sequences training-feature-classifier/ref-seqs.qza \
	--o-oriented-seqs training-feature-classifier/oriented-query-sequences.qza \
	--o-unmatched-seqs training-feature-classifier/unmatched-sequences.qza

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
Saved FeatureData[Sequence] to: training-feature-classifier/oriented-query-sequences.qza
Saved FeatureData[Sequence] to: training-feature-classifier/unmatched-sequences.qza


In [3]:
qiime feature-classifier classify-sklearn \
  --i-classifier training-feature-classifier/classifier.qza \
  --i-reads training-feature-classifier/oriented-query-sequences.qza\
  --o-classification training-feature-classifier/taxonomy.qza

qiime metadata tabulate \
  --m-input-file training-feature-classifier/taxonomy.qza \
  --o-visualization training-feature-classifier/taxonomy.qzv

Plugin error from feature-classifier:

  empty reads input

Debug info has been saved to /var/folders/4f/j976k8fj2yg2v5yjnj1tbkdr0000gs/T/qiime2-q2cli-err-0v87m0ze.log
Saved Visualization to: training-feature-classifier/taxonomy.qzv




Differential abundance testing with ANCOM

ANCOM can be applied to identify features that are differentially abundant (i.e. present in different abundances) across sample groups. As with any bioinformatics method, you should be aware of the assumptions and limitations of ANCOM before using it. We recommend reviewing the ANCOM paper before using this method.

Note

Differential abundance testing in microbiome analysis is an active area of research. There are two QIIME 2 plugins that can be used for this: q2-gneiss and q2-composition. This section uses q2-composition, but there is another tutorial which uses gneiss on a different dataset if you are interested in learning more.

ANCOM is implemented in the q2-composition plugin. ANCOM assumes that few (less than about 25%) of the features are changing between groups. If you expect that more features are changing between your groups, you should not use ANCOM as it will be more error-prone (an increase in both Type I and Type II errors is possible). Because we expect a lot of features to change in abundance across body sites, in this tutorial we’ll filter our full feature table to only contain one sample type. We’ll then apply ANCOM to determine which, if any, sequence variants and genera are differentially abundant across each sample type of the different locations.

We’ll start by creating a feature table that contains only the nest samples. (To learn more about filtering, see the Filtering Data tutorial.) 

Videos explaining more about differential abundance testing and ANCOM: https://www.youtube.com/watch?v=9AmM-BHYxJo&ab_channel=QIIME2 and https://www.youtube.com/watch?v=A6o2nOnDsJU&ab_channel=QIIME2


Differential abundance testing helps us to identify microbes that could be explaining the patterns that we observe in alpha and beta diversity e.g. identifying which microbes have differing abundances between samples.

Basically looks at changes in ratios between features in different samples. 

Because each sample has a particular sequencing depth, a change in ratio between two features (e.g a more negative ratio) could wither mean that one feature has increased or the other has decreased. We can't really tell this apart. But ANCOM can do all of the comparisons between features. Then we can determine which of these is more significant. We can look at the feature that has the most significant changes with other features, and this is likely to be the differentially abundant feature. The number of signicant edges is the W score for that feature. W=5 is the theoretical maximum for an example with 6 features, but it could be less than this. We want to look at the distribution of these W scores. We expect that the distribution will be bimodal - some features will have a high W and some will have a low W. This is how we identify which features are differentially abundant and in what direction. 


NOTE:

As in mice pd tutorial,

You might also want to filter features to exclude:

e.g. feature table 2k where each of the samples have at least 2000 features

e.g. additional refinement which drops features which are seen <50 times or in <4 samples 

We might want to do this because it's more useful if features are actually present in multiple samples otherwise they cant really be differentially abundant. They will either be rare members or just noise so not very important.

To check the distribution of sample frequencies check figures/sample-frequencies.pdf

In [109]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2


In [113]:
cd figures/differential-abundance

In [73]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table table_2k.qza

Saved FeatureTable[Frequency] to: ./table_2k.qza


In [99]:
qiime feature-table filter-samples \
  --i-table deciduous-ant-table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table  deciduous-ant-table_2k.qza

Saved FeatureTable[Frequency] to: deciduous-ant-table_2k.qza


In [100]:
qiime feature-table filter-samples \
  --i-table coniferous-ant-table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table  coniferous-ant-table_2k.qza

Saved FeatureTable[Frequency] to: coniferous-ant-table_2k.qza


In [101]:
qiime feature-table filter-samples \
  --i-table deciduous-nest-table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table  deciduous-nest-table_2k.qza

Saved FeatureTable[Frequency] to: deciduous-nest-table_2k.qza


In [110]:
qiime feature-table filter-samples \
  --i-table coniferous-nest-table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table  coniferous-nest-table_2k.qza

Saved FeatureTable[Frequency] to: coniferous-nest-table_2k.qza


Now have a look at the taxonomy barplot:

In [76]:
qiime taxa barplot \
  --i-table ./table_2k.qza \
  --i-taxonomy  ../training-feature-classifier/taxonomy.qza \
  --m-metadata-file ../sample_metadata.tsv \
  --o-visualization ./taxa_barplot_2k.qzv

Saved Visualization to: ./taxa_barplot_2k.qzv


In [67]:
qiime tools view taxa_barplot_2k.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [ ]:
qiime feature-table filter-features \
  --i-table table_2k.qza \
  --p-min-frequency 50 \
  --p-min-samples 4 \
  --o-filtered-table table_2k_abund.qza

In [102]:
qiime feature-table filter-features \
  --i-table deciduous-ant-table_2k.qza \
  --p-min-frequency 50 \
  --p-min-samples 4 \
  --o-filtered-table deciduous-ant-table_2k_abund.qza

Saved FeatureTable[Frequency] to: deciduous-ant-table_2k_abund.qza


In [103]:
qiime feature-table filter-features \
  --i-table coniferous-ant-table_2k.qza \
  --p-min-frequency 50 \
  --p-min-samples 4 \
  --o-filtered-table coniferous-ant-table_2k_abund.qza

Saved FeatureTable[Frequency] to: coniferous-ant-table_2k_abund.qza


In [104]:
qiime feature-table filter-features \
  --i-table deciduous-nest-table_2k.qza \
  --p-min-frequency 50 \
  --p-min-samples 4 \
  --o-filtered-table deciduous-nest-table_2k_abund.qza

Saved FeatureTable[Frequency] to: deciduous-nest-table_2k_abund.qza


In [111]:
qiime feature-table filter-features \
  --i-table coniferous-nest-table_2k.qza \
  --p-min-frequency 50 \
  --p-min-samples 4 \
  --o-filtered-table coniferous-nest-table_2k_abund.qza

Saved FeatureTable[Frequency] to: coniferous-nest-table_2k_abund.qza


In [80]:
qiime composition add-pseudocount \
  --i-table ./table_2k_abund.qza \
  --o-composition-table ./table2k_abund_comp.qza

Saved FeatureTable[Composition] to: ./table2k_abund_comp.qza


In [106]:
qiime composition add-pseudocount \
  --i-table deciduous-ant-table_2k_abund.qza \
  --o-composition-table deciduous-ant-table2k_abund_comp.qza

Saved FeatureTable[Composition] to: deciduous-ant-table2k_abund_comp.qza


In [107]:
qiime composition add-pseudocount \
  --i-table coniferous-ant-table_2k_abund.qza \
  --o-composition-table coniferous-ant-table2k_abund_comp.qza

Saved FeatureTable[Composition] to: coniferous-ant-table2k_abund_comp.qza


In [108]:
qiime composition add-pseudocount \
  --i-table deciduous-nest-table_2k_abund.qza \
  --o-composition-table deciduous-nest-table2k_abund_comp.qza 

Saved FeatureTable[Composition] to: deciduous-nest-table2k_abund_comp.qza


In [112]:
qiime composition add-pseudocount \
  --i-table coniferous-nest-table_2k_abund.qza \
  --o-composition-table coniferous-nest-table2k_abund_comp.qza 

Saved FeatureTable[Composition] to: coniferous-nest-table2k_abund_comp.qza


In [146]:
qiime composition ancom \
  --i-table ./table2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column nest-no \
  --o-visualization nest-number/ancom_nest-no.qzv


Aborted!


In [44]:
qiime tools view nest-number/ancom_nest-no.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

For the volcano plot, you are looking at the W statistic on the y-axis, and the F-score on the x-axis. So basically the x-axis is summarizing the effect size difference of the given species between your treatment groups, and the y-axis is the strength of the ANCOM test statistic.

What you want to get out of this sort of plot are the ASVs with a high F-score and a high W-statistic – in other words points that are close to the the top right corner. These indicate that an ASV is suspected to be truly different across the groups.

587f8fa74c70d842e9ff2af0bd151e96 has an F-score (clr) of 70 - the furthest in the top right.

This has been classified as: d__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales (taxonomy.tsv)

The next highest F-score (clr) is 2cf75e9f290ebb1c2316be1b719ed41f which is classified as d__Bacteria; p__Firmicutes; c__Bacilli

In [90]:
cd /Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/figures/differential-abundance

In [2]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2


In [87]:
mkdir location

In [93]:
qiime composition ancom \
  --i-table ./table2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization location/ancom_location.qzv

Saved Visualization to: location/ancom_location.qzv


In [114]:
qiime tools view location/ancom_location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

The most top right feature in the volcano plot was d8304d608a24252539009bce47d9c579

This was classified as d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia-Shigella; s__Escherichia_coli

After this was 2bb445f9e9fbd2b77985e53e5b20afdb, classified as d__Bacteria

In [95]:
mkdir tree-type

In [96]:
qiime composition ancom \
  --i-table ./table2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom_tree-type.qzv

Saved Visualization to: tree-type/ancom_tree-type.qzv


In [131]:
qiime tools view tree-type/ancom_tree-type.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

clr 64.9 = dacb187e824b4cdcacbdc1409c7d3591 = d__Bacteria

clr 60.5 = d8304d608a24252539009bce47d9c579 = d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia-Shigella; s__Escherichia_coli

clr 56.9 = 313465e2af6fad406d22a10504e6878f = d__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales

clr 53.7 = 485a8e71354e73ebadebf34e45c07388 = d__Bacteria; p__Firmicutes; c__Bacilli

clr 47.4 = 8a3ae67fed7099b9f0b6479e62cb5174 = d__Bacteria

In [99]:
mkdir sample-type

In [100]:
qiime composition ancom \
  --i-table ./table2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column sample-type \
  --o-visualization sample-type/ancom_sample-type.qzv

Saved Visualization to: sample-type/ancom_sample-type.qzv


In [45]:
qiime tools view sample-type/ancom_sample-type.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

clr 1297 = 63da753bf564b310b04e5e32d75d8152 =

In [103]:
mkdir sample-id

In [104]:
qiime composition ancom \
  --i-table ./table2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column sample-id \
  --o-visualization sample-id/ancom_sample-id.qzv

Usage: qiime composition ancom [OPTIONS]

  Apply Analysis of Composition of Microbiomes (ANCOM) to identify features
  that are differentially abundant across groups.

Inputs:
  --i-table ARTIFACT FeatureTable[Composition]
                       The feature table to be used for ANCOM computation.
                                                                    [required]
Parameters:
  --m-metadata-file METADATA
  --m-metadata-column COLUMN  MetadataColumn[Categorical]
                       The categorical sample metadata column to test for
                       differential abundance across.               [required]
  --p-transform-function TEXT Choices('sqrt', 'log', 'clr')
                       The method applied to transform feature values before
                       generating volcano plots.              [default: 'clr']
  --p-difference-function TEXT Choices('mean_difference', 'f_statistic')
                       The method applied to visualize fold difference in
       

: 1

Cant use sample ID as not categorical column?

Just ant samples:

In [107]:
qiime feature-table filter-samples \
  --i-table table_2k_abund.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --p-where "[sample-type]='ant'" \
  --o-filtered-table ant-table_2k_abund.qza

Saved FeatureTable[Frequency] to: ant-table_2k_abund.qza


ANCOM operates on a FeatureTable[Composition] QIIME 2 artifact, which is based on frequencies of features on a per-sample basis, but cannot tolerate frequencies of zero. To build the composition artifact, a FeatureTable[Frequency] artifact must be provided to add-pseudocount (an imputation method), which will produce the FeatureTable[Composition] artifact.

In [108]:
qiime composition add-pseudocount \
  --i-table ant-table_2k_abund.qza \
  --o-composition-table ant-table_2k_abund-comp.qza

Saved FeatureTable[Composition] to: ant-table_2k_abund-comp.qza


We can then run ANCOM on the subject column to determine what features differ in abundance across the samples of the different locations. Note: this can take a long time. Took about 4 hours for ant comp-table and location metadata column in attempt 1, much much quicker in attempt 2 <30 mins.

In [109]:
qiime composition ancom \
  --i-table ant-table_2k_abund-comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization location/ancom-ant-location.qzv

Saved Visualization to: location/ancom-ant-location.qzv


In [3]:
cd figures/differential-abundance/

In [4]:
qiime tools view location/ancom-ant-location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

clr 77.8 = 2bb445f9e9fbd2b77985e53e5b20afdb = d__Bacteria

clr 72.7 = 76d1bd5834bc0b97db25f8a7125d4fc4 = d__Bacteria; p__Firmicutes; c__Bacilli

clr 47.9 = eefc24ff09351ce9fd0f4f27ea924662 = d__Bacteria

In [133]:
qiime composition ancom \
  --i-table ant-table_2k_abund-comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom-ant-tree-type.qzv

Saved Visualization to: tree-type/ancom-ant-tree-type.qzv


In [6]:
qiime tools view tree-type/ant/ancom-ant-tree-type.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [148]:
qiime composition ancom \
  --i-table ant-table_2k_abund-comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column nest-no \
  --o-visualization nest-number/ancom-ant-nest-no.qzv

Saved Visualization to: nest-number/ancom-ant-nest-no.qzv


In [50]:
qiime tools view nest-number/ancom-ant-nest-no.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Now looking at the ant samples, comparing the locations which have both decidious and coniferous nests:

These included Wyre Forest and Dymock Wood
Deciduous nests:  
WF1, WF3, WF4
DW1, DW3, DW6

Coniferous nests:
WF5, WF6, WF7
DW2, DW4, DW5


In [26]:
qiime feature-table filter-samples \
  --i-table table_2k_abund.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --p-where "[sample-type]='ant' AND [location]='Dymock Wood'" \
  --o-filtered-table ant-Dymock-Wood-table_2k_abund.qza

Saved FeatureTable[Frequency] to: ant-Dymock-Wood-table_2k_abund.qza


Adding the pseudocount:

In [27]:
qiime composition add-pseudocount \
  --i-table ant-Dymock-Wood-table_2k_abund.qza \
  --o-composition-table ant-Dymock-Wood-table_2k_abund_comp.qza

Saved FeatureTable[Composition] to: ant-Dymock-Wood-table_2k_abund_comp.qza


Visualise the analysis:

In [28]:
qiime composition ancom \
  --i-table ant-Dymock-Wood-table_2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom-tree-type-ant-Dymock-Wood.qzv

Saved Visualization to: tree-type/ancom-tree-type-ant-Dymock-Wood.qzv


In [8]:
qiime tools view tree-type/ant/ancom-tree-type-ant-Dymock-Wood.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Only one differentially abundant feature

665793d42b5fef3492e0e09a394030ee = d__Bacteria

Blasted rRNA bacteria and returned a list of 	Streptomyces sp.

Now to took at nest samples for Dymock Wood:

In [30]:
qiime feature-table filter-samples \
  --i-table table_2k_abund.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --p-where "[sample-type]='nest' AND [location]='Dymock Wood'" \
  --o-filtered-table nest-Dymock-Wood-table_2k_abund.qza

Saved FeatureTable[Frequency] to: nest-Dymock-Wood-table_2k_abund.qza


Adding the pseudocount:

In [31]:
qiime composition add-pseudocount \
  --i-table nest-Dymock-Wood-table_2k_abund.qza \
  --o-composition-table nest-Dymock-Wood-table_2k_abund_comp.qza

Saved FeatureTable[Composition] to: nest-Dymock-Wood-table_2k_abund_comp.qza


In [ ]:
Visualise the analysis:

In [32]:
qiime composition ancom \
  --i-table nest-Dymock-Wood-table_2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom-tree-type-nest-Dymock-Wood.qzv

Saved Visualization to: tree-type/ancom-tree-type-nest-Dymock-Wood.qzv


In [52]:
qiime tools view tree-type/ancom-tree-type-nest-Dymock-Wood.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Now for the other mixed location: Wyre Forest

Starting with ant samples:

In [35]:
qiime feature-table filter-samples \
  --i-table table_2k_abund.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --p-where "[sample-type]='ant' AND [location]='Wyre Forest'" \
  --o-filtered-table ant-Wyre-Forest-table_2k_abund.qza

Saved FeatureTable[Frequency] to: ant-Wyre-Forest-table_2k_abund.qza


In [36]:
qiime composition add-pseudocount \
  --i-table ant-Wyre-Forest-table_2k_abund.qza \
  --o-composition-table ant-Wyre-Forest-table_2k_abund_comp.qza

Saved FeatureTable[Composition] to: ant-Wyre-Forest-table_2k_abund_comp.qza


In [37]:
qiime composition ancom \
  --i-table ant-Wyre-Forest-table_2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom-tree-type-ant-Wyre-Forest.qzv

Saved Visualization to: tree-type/ancom-tree-type-ant-Wyre-Forest.qzv


In [53]:
qiime tools view tree-type/ancom-tree-type-ant-Wyre-Forest.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Now for Wyre Forest nest samples:

In [39]:
qiime feature-table filter-samples \
  --i-table table_2k_abund.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --p-where "[sample-type]='nest' AND [location]='Wyre Forest'" \
  --o-filtered-table nest-Wyre-Forest-table_2k_abund.qza

Saved FeatureTable[Frequency] to: nest-Wyre-Forest-table_2k_abund.qza


In [ ]:
qiime composition add-pseudocount \
  --i-table nest-Wyre-Forest-table_2k_abund.qza \
  --o-composition-table nest-Wyre-Forest-table_2k_abund_comp.qza

Saved FeatureTable[Composition] to: nest-Wyre-Forest-table_2k_abund_comp.qza


In [42]:
qiime composition ancom \
  --i-table nest-Wyre-Forest-table_2k_abund_comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom-tree-type-nest-Wyre-Forest.qzv

Saved Visualization to: tree-type/ancom-tree-type-nest-Wyre-Forest.qzv


In [43]:
qiime tools view tree-type/ancom-tree-type-nest-Wyre-Forest.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Just nest samples:

Filter nest samples:

In [113]:
qiime feature-table filter-samples \
  --i-table table_2k_abund.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --p-where "[sample-type]='nest'" \
  --o-filtered-table nest-table_2k_abund.qza

Saved FeatureTable[Frequency] to: nest-table_2k_abund.qza


Add pseudocount to deal with 0s:

In [114]:
qiime composition add-pseudocount \
  --i-table nest-table_2k_abund.qza \
  --o-composition-table nest-table_2k_abund-comp.qza

Saved FeatureTable[Composition] to: nest-table_2k_abund-comp.qza


Run ANCOM:

In [116]:
qiime composition ancom \
  --i-table nest-table_2k_abund-comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization location/ancom-nest-location.qzv

Saved Visualization to: location/ancom-nest-location.qzv


In [54]:
qiime tools view location/ancom-nest-location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

clr 753 = f63eac3240d8794b40bc751d4ba8360a = d__Bacteria 

In [119]:
qiime composition ancom \
  --i-table nest-table_2k_abund-comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column tree-type \
  --o-visualization tree-type/ancom-nest-tree-type.qzv

Saved Visualization to: tree-type/ancom-nest-tree-type.qzv


In [55]:
qiime tools view tree-type/ancom-nest-tree-type.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

clr 3.44 = 25b06de1105b898dfc96999bd1aeb98a = d__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Marinococcaceae; g__Marinococcus

Note: can't do nest number as they are all unique and therefore you can't compare differences, you cant do sample type because as we've already filtered for nest they are all the same so there are no 'between' variances

In [ ]:
qiime composition ancom \
  --i-table nest-table_2k_abund-comp.qza \
  --m-metadata-file ../../sample_metadata.tsv \
  --m-metadata-column nest-no \
  --o-visualization nest-number/ancom-nest-nest-no.qzv

In [120]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/figures/differential-abundance


In [117]:
mkdir location/coniferous
mkdir location/deciduous

In [122]:
cd ..

In [123]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2/figures


In [124]:
cd ..

In [125]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis-attempt2


In [128]:
qiime composition ancom \
  --i-table coniferous-ant-table2k_abund_comp.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization figures/differential-abundance/location/coniferous/ancom-coniferous-ant.qzv

Saved Visualization to: figures/differential-abundance/location/coniferous/ancom-coniferous-ant.qzv


In [130]:
qiime tools view figures/differential-abundance/location/coniferous/ancom-coniferous-ant.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [134]:
qiime composition ancom \
  --i-table deciduous-ant-table2k_abund_comp.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization figures/differential-abundance/location/deciduous/ancom-deciduous-ant.qzv

Saved Visualization to: figures/differential-abundance/location/deciduous/ancom-deciduous-ant.qzv


In [135]:
qiime tools view figures/differential-abundance/location/deciduous/ancom-deciduous-ant.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [136]:
qiime composition ancom \
  --i-table deciduous-nest-table2k_abund_comp.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization figures/differential-abundance/location/deciduous/ancom-deciduous-nest.qzv

Saved Visualization to: figures/differential-abundance/location/deciduous/ancom-deciduous-nest.qzv


In [141]:
qiime tools view figures/differential-abundance/location/deciduous/ancom-deciduous-nest.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [138]:
qiime composition ancom \
  --i-table coniferous-nest-table2k_abund_comp.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization figures/differential-abundance/location/coniferous/ancom-coniferous-nest.qzv

Saved Visualization to: figures/differential-abundance/location/coniferous/ancom-coniferous-nest.qzv


In [140]:
qiime tools view figures/differential-abundance/location/coniferous/ancom-coniferous-nest.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Specify W cutoff for anacom?
https://forum.qiime2.org/t/specify-w-cutoff-for-anacom/1844?u=mortonjt

Differential abundance is a super touchy topic – there are hundreds of tools out there to do this, and all of them have their own set of assumptions and weaknesses.

The threshold for the W value is automatically determined (see ANCOM paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4450248/), so the hypothesis rejection process is a bit hidden from the user. So no, you cannot set threshold for the W-value.

We’re also often interested in performing a differential abundance test at a specific taxonomic level. To do this, we can collapse the features in our FeatureTable[Frequency] at the taxonomic level of interest, and then re-run the above steps. 

Ancom statistical results table shows the features that reject the null hypothesis, which means that they are differentially abundant

Percentile abundance of features by group table shows the percentile of the features in different groups. Numbers are the number of reads of those features. see explaiation: https://forum.qiime2.org/t/interpreting-values-from-an-ancom-percentile-abundance-table/1497/14?u=nicholas_bokulich

"The minimum value (denoted by Percentile 0) is 1 (which in your case is actually zero since you added a pseudocount in ANCOM). And your maximum (denoted by Percentile 100) is 57. The median (denoted by Percentile 50) is 23. Percentiles 25 and 75 just denote quartiles.

You mentioned that the bacteria X (some taxon) percentiles for your treatment1 category are as follows:

Min: 1
25th percentile: 84
50th percentile (median): 283
75th percentile: 825
Max: 3645

The interpretations of these values are as follows:

Min: in the table provided as input to ancom, of the samples in the treatment1 group, in the sample with the lowest count of sequences assigned to bacteria X, one sequence was observed that was ultimately assigned the taxon bacteria X.

25th percentile: In 25% of the samples in the treatment1 group, 84 or fewer sequences were observed that were ultimately assigned the taxon bacteria X.

50th percentile (median): In half of the samples in the treatment1 group, 283 or fewer sequences were observed that were ultimately assigned the taxon bacteria X.

75th percentile: In 75% of the samples in the treatment1 group, 825 or fewer sequences were observed that were ultimately assigned the taxon bacteria X.

Max: Of the samples in the treatment1 group, in the sample with the highest count of sequences assigned to bacteria X, 3645 sequences were observed that were ultimately assigned the taxon bacteria X."

You can visualise this as a box plot as shown in the link.

In [86]:
qiime taxa collapse \
  --i-table ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --p-level 7 \
  --o-collapsed-table ant-table-l7.qza

qiime composition add-pseudocount \
  --i-table ant-table-l7.qza \
  --o-composition-table comp-ant-table-l7.qza

qiime composition ancom \
  --i-table comp-ant-table-l7.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization l7-ancom-location.qzv

Saved FeatureTable[Frequency] to: ant-table-l7.qza
Saved FeatureTable[Composition] to: comp-ant-table-l7.qza
Saved Visualization to: l7-ancom-location.qzv


In [115]:
qiime tools view l7-ancom-location.qzv

Usage: qiime tools view [OPTIONS] VISUALIZATION

  Displays a QIIME 2 Visualization until the command exits. To open a QIIME
  2 Visualization so it can be used after the command exits, use 'qiime
  tools extract'.

Options:
  --index-extension TEXT  The extension of the index file that should be
                          opened.                              [default: html]
  --help                  Show this message and exit.

                    There was a problem with the command:                     
 (1/1) Invalid value for 'VISUALIZATION': File 'l7-ancom-location.qzv' does
  not exist.


: 1

In [78]:
qiime taxa collapse \
  --i-table ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table ant-table-l6.qza

qiime composition add-pseudocount \
  --i-table ant-table-l6.qza \
  --o-composition-table comp-ant-table-l6.qza

qiime composition ancom \
  --i-table comp-ant-table-l6.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization l6-ancom-location.qzv

Saved FeatureTable[Frequency] to: ant-table-l6.qza
Saved FeatureTable[Composition] to: comp-ant-table-l6.qza
Saved Visualization to: l6-ancom-location.qzv


In [79]:
qiime tools view l6-ancom-location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [80]:
qiime taxa collapse \
  --i-table ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --p-level 5 \
  --o-collapsed-table ant-table-l5.qza

qiime composition add-pseudocount \
  --i-table ant-table-l5.qza \
  --o-composition-table comp-ant-table-l5.qza

qiime composition ancom \
  --i-table comp-ant-table-l5.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization l5-ancom-location.qzv

Saved FeatureTable[Frequency] to: ant-table-l5.qza
Saved FeatureTable[Composition] to: comp-ant-table-l5.qza
Saved Visualization to: l5-ancom-location.qzv


In [81]:
qiime tools view l5-ancom-location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [84]:
qiime taxa collapse \
  --i-table ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --p-level 4 \
  --o-collapsed-table ant-table-l4.qza

qiime composition add-pseudocount \
  --i-table ant-table-l4.qza \
  --o-composition-table comp-ant-table-l4.qza

qiime composition ancom \
  --i-table comp-ant-table-l4.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization l4-ancom-location.qzv

Saved FeatureTable[Frequency] to: ant-table-l4.qza
Saved FeatureTable[Composition] to: comp-ant-table-l4.qza
Saved Visualization to: l4-ancom-location.qzv


In [88]:
qiime tools view l4-ancom-location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [82]:
qiime taxa collapse \
  --i-table ant-table.qza \
  --i-taxonomy training-feature-classifier/taxonomy.qza \
  --p-level 1 \
  --o-collapsed-table ant-table-l1.qza

qiime composition add-pseudocount \
  --i-table ant-table-l1.qza \
  --o-composition-table comp-ant-table-l1.qza

qiime composition ancom \
  --i-table comp-ant-table-l1.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization l1-ancom-location.qzv

Saved FeatureTable[Frequency] to: ant-table-l1.qza
Saved FeatureTable[Composition] to: comp-ant-table-l1.qza
Saved Visualization to: l1-ancom-location.qzv


In [83]:
qiime tools view l1-ancom-location.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Why are there eukaryota and archaea in my 16S sequences?

Remember the SILVA SSU contains not only Bacterial, Acrachaeal, and Eukaryal 16S rRNA gene sequences, but also contains the Eukaryal 18S rRNA gene sequences too (which are longer). That is, the 18S rRNA gene is a cystolic homologue of the 16S rRNA gene.

https://forum.qiime2.org/t/silva138-16s-database-contains-eukaryota-after-filtering/22929/2 


Question

Which genera differ in abundance across subject? In which subject is each genus more abundant?

Now for nest samples:

In [106]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample_metadata.tsv \
  --p-where "[sample-type]='nest'" \
  --o-filtered-table nest-table.qza

Saved FeatureTable[Frequency] to: nest-table.qza


In [2]:
qiime composition add-pseudocount \
  --i-table nest-table.qza \
  --o-composition-table nest-comp-table.qza

Saved FeatureTable[Composition] to: nest-comp-table.qza


In [2]:
qiime composition ancom \
  --i-table nest-comp-table.qza \
  --m-metadata-file sample_metadata.tsv \
  --m-metadata-column location \
  --o-visualization nest-ancom-location.qzv


Aborted!


In [2]:
qiime composition ancom --i-table nest-comp-table.qza --m-metadata-file sample_metadata.tsv --m-metadata-column location --o-visualization nest-ancom-location.qzv


Aborted!


In [1]:
pwd

/Users/d1794974/Documents/PhD/Wood_ants/Amplicon_sequencing/qiime2-analysis


Export Taxonomy as TSV

NOTE: where does taxonomy.qza come from? 

In [103]:
qiime tools export \
--input-path training-feature-classifier/taxonomy.qza \
--output-path figures

Exported training-feature-classifier/taxonomy.qza as TSVTaxonomyDirectoryFormat to directory figures


Delete the header lines of the .tsv files

In [104]:
sed '1d' figures/taxonomy.tsv > figures/taxonomy_noHeader.tsv
sed '1d' figures/feature-table.tsv > figures/feature-table_noHeader.tsv